# Example 4: Video analysis - predator prey interactions

[ADD TEXT]

In [ ]:
import phenopype as pp
import trackpy as tp #can be installed using pip

In [ ]:
video_path =  r"images/isopods_fish.mp4"
out_dir =  r"../_temp/video"

In [ ]:
# draw masks
mt = pp.motion_tracker(video_path)
mt.video_output(save_suffix="v1", dirpath=out_dir)

pp.preprocessing.create_mask(mt, label="full arena")

# create fish method
fish = pp.tracking_method(label="fish",  blur=15, threshold=100, remove_shadows=True, 
                          min_length=50, max_length=250, overlay_colour="red",
                          mode="single") 
isopods = pp.tracking_method(label="isopod", blur=9, threshold=80, remove_shadows=False, 
                            min_length=8, max_length=30, overlay_colour="green", 
                            operations=["diameter", "area","grayscale","grayscale_background","bgr"]) 

# run motion tracker
mt.motion_detection(threshold=10, skip=5, start_after=1, history=500, 
                    methods=[fish, isopods], detect_shadows=True, mode="MOG")
coordinates = mt.run_tracking(show="overlay", weight=0.5, return_df = True)

# save output
coordinates.to_csv(os.path.join(out_dir, mt.name + "_coordinates.csv"), sep=',')
coordinates = pd.read_csv(os.path.join(out_dir, mt.name + "_coordinates.csv"))

